In [4]:
!pip install matplotlib

  Using cached importlib_resources-6.4.0-py3-none-any.whl (38 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
  Using cached contourpy-1.2.1-cp39-cp39-win_amd64.whl (182 kB)
  Using cached kiwisolver-1.4.5-cp39-cp39-win_amd64.whl (56 kB)


In [2]:
!pip install torch torchvision torchaudio

In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np

transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [24]:
# Данные организованы в виде папок с изображениями, где каждая папка соответствует классу
train_data_dir = 'C:/Users/nasty/Desktop/train'
test_data_dir = 'C:/Users/nasty/Desktop/test'

# Загрузка тренировочного набора данных
trainset = ImageFolder(root=train_data_dir, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

# Загрузка тестового набора данных
testset = ImageFolder(root=test_data_dir, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Список классов (папок) в вашем наборе данных
classes = trainset.classes

In [28]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Загрузка предобученной модели ResNet18
net = models.resnet18(pretrained=True)

# Замена последнего полносвязного слоя, чтобы он соответствовал количеству классов в наборе данных
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, len(classes))

# Перенос модели на GPU, если доступно
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [20]:
for epoch in range(10):  # количество эпох
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход, обратный проход, оптимизация
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Вывод статистики
        running_loss += loss.item()
        if i % 200 == 199:  # каждые 200 мини-батчей
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 1.011
[2,   200] loss: 0.579
[3,   200] loss: 0.471
[4,   200] loss: 0.428
[5,   200] loss: 0.403
[6,   200] loss: 0.364
[7,   200] loss: 0.324


KeyboardInterrupt: 

In [25]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 100 %


In [26]:
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(len(classes)):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of apple_pie : 100 %
Accuracy of baby_back_ribs : 66 %
Accuracy of baklava : 100 %
Accuracy of beef_carpaccio : 100 %
Accuracy of beef_tartare : 100 %
Accuracy of beet_salad : 100 %
Accuracy of beignets : 100 %
